In [1]:
library(edgeR)
library(ggplot2)
library(glue)

Loading required package: limma

Warning message:
“package ‘ggplot2’ was built under R version 4.0.0”


## 1. load in data

In [2]:
raw_filename            = '../data/16p12_lcl_gene_reads_underscores.gct'
pheno_filename          = '../data/pheno_final.tsv'
mapping_filename        = '../family_based_analysis3/gene_map.tsv'

In [3]:
mapping             = read.table(mapping_filename, sep='\t', header=TRUE, stringsAsFactors = F)
rownames(mapping)   = mapping$Name

In [4]:
pheno                   = read.table(pheno_filename, sep='\t', header=TRUE)
rownames(pheno)         = pheno$sample

In [5]:
rawdf               = read.table(raw_filename, sep='\t', header=TRUE)
rownames(rawdf)     = rawdf$Name
rawdf$Name          = NULL
gencode2ensembl = function(s) {
    return(strsplit(s, '.', fixed=T)[[1]][1])
}

rownames(rawdf) = unlist(lapply(rownames(rawdf), gencode2ensembl))

In [7]:
# Load in sex genes
# low_expressed_genes = scan('../family_based_analysis3/genes.low_expression', what="", sep="\n")
sex_genes = scan('../family_based_analysis3/gtex_filter_sex_diff2.list', what="", sep="\n")
xgenes = as.character(mapping[mapping$chromosome == 'X' ,]$ensembl)
ygenes = as.character(mapping[mapping$chromosome == 'Y' ,]$ensembl)
# length(low_expressed_genes)
length(xgenes)
length(ygenes)
length(sex_genes)

[1] 2358

[1] 491

[1] 340

In [8]:
# Filter out sex genes
dim(rawdf)
rawdf=rawdf[!(rownames(rawdf) %in% xgenes),]
dim(rawdf)
rawdf=rawdf[!(rownames(rawdf) %in% ygenes),]
dim(rawdf)
rawdf=rawdf[!(rownames(rawdf) %in% sex_genes),]
dim(rawdf)
rawmat              = as.matrix(rawdf)

[1] 56202    96

[1] 53844    96

[1] 53353    96

[1] 53013    96

In [9]:
rawmat          = as.matrix(rawdf)
pheno = pheno[colnames(rawmat),]

In [10]:
contrast_map = function(design, group) {
    group = paste0('group', group)
    pos = (1:length(colnames(design)))[colnames(design) == group]
    return(pos)
}

In [11]:
group         = pheno$status3
family        = pheno$family

In [12]:
y = DGEList(counts=rawmat, group=group)
keep = filterByExpr(y)
y = y[keep,,keep.lib.sizes=FALSE]
y = calcNormFactors(y)

In [13]:
write.table(names(keep[keep]), 'output/no_sex/keep_carrier_non_carrier.txt', 
            row.names = F, col.names = F, quote = F)


In [16]:
length(keep[keep])

[1] 27079

In [17]:
#Create the design matrix for all samples
design        = model.matrix(~0+group+family)
rownames(design) = colnames(y)
#Estimate dispersion of variances along model
y = estimateDisp(y, design, robust=TRUE)
#Model the quasi-lielihood dispersions of variation within model
fit = glmQLFit(y, design, robust=TRUE)

In [18]:
contrast_map = function(design, group) {
    group = paste0('group', group)
    pos = (1:length(colnames(design)))[colnames(design) == group]
    return(pos)
}

In [19]:
contrast = numeric(length(colnames(design)))
contrast[contrast_map(design, 'non_carrier')] = -1
contrast[contrast_map(design, 'carrier')] = 1

In [21]:
qlf = glmQLFTest(fit,contrast=contrast)

In [22]:
topdf = topTags(qlf, n=56202, p.value=0.05)$table

In [23]:
topdf['gene'] = mapping[rownames(topdf), 'Description']
topdf = topdf[c('gene', 'logFC', 'logCPM', 'F', 'PValue', 'FDR')]

In [24]:
diff_save = cbind(rownames(topdf), topdf)
colnames(diff_save)[1] = "ensembl"
write.table(diff_save, 'output/no_sex/carrier_non_carrier.tsv', sep='\t', row.names=F, col.names=T)




In [25]:
for (excl_sub in unique(pheno$subject)) {
    excl_sub
    excl_pheno = pheno[pheno$subject != excl_sub,]
    samples = excl_pheno$sample
    excl_rawmat = rawmat[,samples]
    
    group         = excl_pheno$status3
    family        = excl_pheno$family
    
    y = DGEList(counts=excl_rawmat, group=group)
    keep = filterByExpr(y)
    write.table(names(keep[keep]), glue('output/no_sex/keep.{excl_sub}.txt'), 
            row.names = F, col.names = F, quote = F)
    y = y[keep,,keep.lib.sizes=FALSE]
    y = calcNormFactors(y)
    
    design        = model.matrix(~0+group+family)
    rownames(design) = colnames(y)
    y = estimateDisp(y, design, robust=TRUE)
    fit = glmQLFit(y, design, robust=TRUE)
    
    contrast = numeric(length(colnames(design)))
    contrast[contrast_map(design, 'non_carrier')] = -1
    contrast[contrast_map(design, 'carrier')] = 1
    
    qlf = glmQLFTest(fit,contrast=contrast)
    topdf = topTags(qlf, n=56202, p.value=0.05)$table
    dim(topdf)
    
    diff_save = cbind(rownames(topdf), topdf)
    colnames(diff_save)[1] = "ensembl"
    outfile = glue('output/no_sex/{excl_sub}.tsv')
    write.table(diff_save, outfile, sep='\t', row.names=F, col.names=T)
}